In [808]:
!pip install unidecode
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install joblib


In [809]:
import pandas as pd
datos = pd.read_csv("/Users/carloscole/Downloads/TablaFinalSinDetalles.csv", encoding = 'utf8', sep=';')

In [810]:
def f_equipo(x):
    return x.split('\n')[-1].split(',')[0]
    
def f_player(x):
    return x.split('\n')[1]

In [811]:
datos['Equipo'] = datos['Player'].map(f_equipo)
datos['Player'] = datos['Player'].map(f_player)


In [812]:
datos.keys()

Index(['Player', '_x', 'Apps_x', 'Mins_x', 'Goals_x', 'Assists_x', 'Yel',
       'Red', 'SpG_x', 'PS%_x', 'AerialsWon', 'MotM', 'Rating_x', '_y',
       'Apps_y', 'Mins_y', 'Tackles', 'Inter', 'Fouls', 'Offsides', 'Clear',
       'Drb_x', 'Blocks', 'OwnG', 'Rating_y', '_x.1', 'Apps_x.1', 'Mins_x.1',
       'Goals_y', 'Assists_y', 'SpG_y', 'KeyP_x', 'Drb_y', 'Fouled', 'Off',
       'Disp', 'UnsTch', 'Rating_x.1', '_y.1', 'Apps_y.1', 'Mins_y.1',
       'Assists', 'KeyP_y', 'AvgP', 'PS%_y', 'Crosses', 'LongB', 'ThrB',
       'Rating_y.1', 'año', 'Equipo'],
      dtype='object')

In [813]:
columnas_eliminar = ['_x','Apps_y', 'Mins_y', 'Goals_y', 'Assists_y', 'SpG_y', 'Rating_x',
                     'Rating_y', '_y', 'KeyP_y', 'PS%_y','_x.1', '_y.1', 'Rating_x.1', 'Rating_y.1',
                    'Apps_x.1','Mins_x.1', 'Apps_y.1', 'Mins_y.1', 'Assists']
datos = datos.drop(columnas_eliminar,axis=1)
datos = datos.rename(columns={'Drb_x': 'DeffDrb'})
datos = datos.rename(columns={'Drb_y': 'OffDrb'})

In [814]:
datos.keys()

Index(['Player', 'Apps_x', 'Mins_x', 'Goals_x', 'Assists_x', 'Yel', 'Red',
       'SpG_x', 'PS%_x', 'AerialsWon', 'MotM', 'Tackles', 'Inter', 'Fouls',
       'Offsides', 'Clear', 'DeffDrb', 'Blocks', 'OwnG', 'KeyP_x', 'OffDrb',
       'Fouled', 'Off', 'Disp', 'UnsTch', 'AvgP', 'Crosses', 'LongB', 'ThrB',
       'año', 'Equipo'],
      dtype='object')

In [815]:
def quitar_x(x):
    return x.split('_')[0]

In [816]:
datos = datos.replace('-', 0)
nuevos_nombres = dict(zip(datos.columns, map(quitar_x, datos.columns)))

# Aplicar el mapeo de nombres a través de rename
datos = datos.rename(columns=nuevos_nombres)


In [817]:
tf_data = pd.read_csv('/Users/carloscole/Downloads/TransferMarket_completo.csv')
tf_data['Anio'] = tf_data['Anio'].astype(str) + '-' + (tf_data['Anio'] + 1).astype(str)
columnas_eliminar = ['Unnamed: 0.1','Unnamed: 0', 'id', 'currentClub']
tf_data = tf_data.drop(columnas_eliminar, axis = 1)
tf_data = tf_data.dropna(subset=['marketValue'], axis=0)
datos = datos[datos['año'] != '2023-2024']

In [818]:
condicion = tf_data['name'].isin(datos['Player'])

# Filtrar tf_data con la condición
resultados = tf_data[~condicion]

# Imprimir los resultados
print(len(resultados))

747


In [819]:

from unidecode import unidecode
from fuzzywuzzy import fuzz
from joblib import Parallel, delayed
import numpy as np

ca = np.array(datos["Player"])
cb = np.array(resultados['name'])

# Umbral para la comparación de similitud
thresh = 80  # Ajusta según tus necesidades

# Función para comparar en paralelo
def parallel_fuzzy_match(idxa, idxb):
    return [ca[idxa], cb[idxb], fuzz.ratio(ca[idxa], cb[idxb])]

# Parallel Code
results = Parallel(n_jobs=-1, verbose=1)(
    delayed(parallel_fuzzy_match)(idx1, idx2)
    for idx1 in range(len(ca))
    for idx2 in range(len(cb))
    if fuzz.ratio(ca[idx1], cb[idx2]) > thresh
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   24.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   54.4s
[Parallel(n_jobs=-1)]: Done 469 out of 469 | elapsed:   56.6s finished


In [820]:
resultados_filtrados_96_99 = [elemento for elemento in results if ((elemento[2] < 100) and (elemento[2] > 95))]
print(resultados_filtrados_96_99)
datos.loc[datos['Player'] == resultados_filtrados_96_99[0][0], 'Player'] = resultados_filtrados_96_99[0][1]
print(datos[datos['Player'] == resultados_filtrados_96_99[0][1]])


[["Mu'nas Dabbur", 'Munas Dabbur', 96], ['Carlos Martín', 'Carlos Marín', 96], ['Carlos Martín', 'Carlos Marín', 96], ['Carlos Martín', 'Carlos Marín', 96], ['Carlos Martín', 'Carlos Marín', 96]]
           Player  Apps  Mins Goals Assists Yel Red SpG PS% AerialsWon  ...  \
454  Munas Dabbur  0(2)    24     0       0   0   0   1  75          0  ...   

    Fouled Off Disp UnsTch AvgP Crosses LongB ThrB        año   Equipo  
454      0   0  0.5      1    8     0.5     0  0.5  2019-2020  Sevilla  

[1 rows x 31 columns]


In [821]:
resultados_filtrados_93_94 = [elemento for elemento in results if ((elemento[2] < 95) and (elemento[2] > 92))]
indices_a_eliminar = [24, 25]
print(resultados_filtrados_93_94)
# Utilizar slicing para eliminar los elementos correspondientes a los índices especificados
resultados_filtrados_93_94 = [elemento for i, elemento in enumerate(resultados_filtrados_94_96) if i not in indices_a_eliminar]
for resultado in resultados_filtrados_93_94:
    datos.loc[datos['Player'] == resultado[0], 'Player'] = resultado[1]

[['Dani Carvajal', 'Daniel Carvajal', 93], ['Dani Carvajal', 'Daniel Carvajal', 93], ['Dani Carvajal', 'Daniel Carvajal', 93], ['Dani Carvajal', 'Daniel Carvajal', 93], ['Íñigo Martínez', 'Iñigo Martínez', 93], ['Íñigo Martínez', 'Iñigo Martínez', 93], ['Íñigo Martínez', 'Iñigo Martínez', 93], ['Íñigo Martínez', 'Iñigo Martínez', 93], ['Jaume Domènech', 'Jaume Doménech', 93], ['Jaume Domènech', 'Jaume Doménech', 93], ['Jaume Domènech', 'Jaume Doménech', 93], ['Jaume Domènech', 'Jaume Doménech', 93], ['Karl Toko-Ekambi', 'Karl Toko Ekambi', 94], ['Xavi Quintillá', 'Xavi Quintillà', 93], ['Javier Ontiveros', 'Javi Ontiveros', 93], ['Javier Ontiveros', 'Javi Ontiveros', 93], ['Javier Ontiveros', 'Javi Ontiveros', 93], ['Marko Dmitrovic', 'Marko Dmitrović', 93], ['Marko Dmitrovic', 'Marko Dmitrović', 93], ['José Luís Morales', 'José Luis Morales', 94], ['Óscar De Marcos', 'Óscar de Marcos', 93], ['Óscar De Marcos', 'Óscar de Marcos', 93], ['Óscar De Marcos', 'Óscar de Marcos', 93], ['Óscar

In [822]:
resultados_filtrados_92 = [elemento for elemento in results if elemento[2] == 92]
print(resultados_filtrados_92)
for resultado in resultados_filtrados_92:
    datos.loc[datos['Player'] == resultado[0], 'Player'] = resultado[1]


[['Adri Embarba', 'Adrián Embarba', 92], ['Adri Embarba', 'Adrián Embarba', 92], ['Okay Yokuslu', 'Okay Yokuşlu', 92], ['Okay Yokuslu', 'Okay Yokuşlu', 92], ['Okay Yokuslu', 'Okay Yokuşlu', 92], ['Okay Yokuslu', 'Okay Yokuşlu', 92], ['Joseba Zaldúa', 'Joseba Zaldua', 92], ['Joseba Zaldúa', 'Joseba Zaldua', 92], ['Joseba Zaldúa', 'Joseba Zaldua', 92], ['Joseba Zaldúa', 'Joseba Zaldua', 92], ['Xabier Etxeita', 'Xabi Etxeita', 92], ['Xabier Etxeita', 'Xabi Etxeita', 92], ['Ronald Araújo', 'Ronald Araujo', 92], ['Ronald Araújo', 'Ronald Araujo', 92], ['Ronald Araújo', 'Ronald Araujo', 92], ['Ronald Araújo', 'Ronald Araujo', 92], ['Kevin Gameiro', 'Kévin Gameiro', 92], ['Kevin Gameiro', 'Kévin Gameiro', 92], ['Édgar Méndez', 'Edgar Méndez', 92], ['Édgar Méndez', 'Edgar Méndez', 92], ['Édgar Méndez', 'Edgar Méndez', 92], ['Ronald Araújo', 'Ronald Araujo', 92], ['Ronald Araújo', 'Ronald Araujo', 92], ['Ronald Araújo', 'Ronald Araujo', 92], ['Ronald Araújo', 'Ronald Araujo', 92], ['Victor Chus

In [823]:
resultados_filtrados_91 = [elemento for elemento in results if elemento[2] == 91]
print(resultados_filtrados_91)
for resultado in resultados_filtrados_91:
    datos.loc[datos['Player'] == resultado[0], 'Player'] = resultado[1]



[['Adrián López', 'Adri López', 91], ['Lago Junior', 'Lago Júnior', 91], ['Lago Junior', 'Lago Júnior', 91], ['Lago Junior', 'Lago Júnior', 91], ['Lago Junior', 'Lago Júnior', 91], ['Íñigo Lekue', 'Iñigo Lekue', 91], ['Íñigo Lekue', 'Iñigo Lekue', 91], ['Íñigo Lekue', 'Iñigo Lekue', 91], ['Íñigo Lekue', 'Iñigo Lekue', 91], ['Ramón Azeez', 'Ramon Azeez', 91], ['Ramón Azeez', 'Ramon Azeez', 91], ['Yeremy Pino', 'Yéremy Pino', 91], ['Yeremy Pino', 'Yéremy Pino', 91], ['Yeremy Pino', 'Yéremy Pino', 91], ['Íñigo Lekue', 'Iñigo Lekue', 91], ['Íñigo Lekue', 'Iñigo Lekue', 91], ['Íñigo Lekue', 'Iñigo Lekue', 91], ['Íñigo Lekue', 'Iñigo Lekue', 91], ['Aleix García', 'Ale García', 91], ['Aleix García', 'Ale García', 91], ['Adrián López', 'Adri López', 91], ['Ramón Azeez', 'Ramon Azeez', 91], ['Ramón Azeez', 'Ramon Azeez', 91], ['Érik Lamela', 'Erik Lamela', 91], ['Érik Lamela', 'Erik Lamela', 91], ['Íñigo Ruíz de Galarreta', 'Iñigo Ruiz de Galarreta', 91], ['Íñigo Ruíz de Galarreta', 'Iñigo Ruiz

In [824]:
resultados_filtrados_90 = [elemento for elemento in results if elemento[2] == 90]
print(resultados_filtrados_90)
datos.loc[datos['Player'] == resultados_filtrados_90[0][0], 'Player'] = resultados_filtrados_90[0][1]
print(datos[datos['Player'] == resultados_filtrados_90[0][1]])

[['Luka Jovic', 'Luka Jović', 90], ['Luka Jovic', 'Luka Jović', 90], ['Luka Jovic', 'Luka Jović', 90], ['Christian Rivera', 'Cristian Rivero', 90], ['Christian Rivera', 'Cristian Rivero', 90], ['Christian Rivera', 'Cristian Rivero', 90], ['Christian Rivera', 'Cristian Rivero', 90], ['Christian Oliva', 'Cristian Olivera', 90], ['Luka Jovic', 'Luka Jović', 90], ['Luka Jovic', 'Luka Jović', 90], ['Luka Jovic', 'Luka Jović', 90], ['Luka Jovic', 'Luka Jović', 90], ['Luka Jovic', 'Luka Jović', 90], ['Luka Jovic', 'Luka Jović', 90]]
          Player   Apps  Mins Goals Assists Yel Red  SpG   PS% AerialsWon  \
405   Luka Jović  4(13)   422     2       1   2   0  0.8  80.6        0.9   
1042  Luka Jović   2(2)   149     0       0   0   0  1.5  84.6        0.8   
1572  Luka Jović  1(14)   360     1       3   1   0  1.1  77.6        0.5   

      ... Fouled  Off Disp UnsTch AvgP Crosses LongB ThrB        año  \
405   ...    0.1  0.2  0.2    0.5  7.9       0   0.2    0  2019-2020   
1042  ...    0.

In [825]:
resultados_filtrados_89 = [elemento for elemento in results if elemento[2] == 89]
print(resultados_filtrados_89)
datos.loc[datos['Player'] == resultados_filtrados_89[-1][0], 'Player'] = resultados_filtrados_89[-1][1]
print(datos[datos['Player'] == resultados_filtrados_89[-1][1]])

[['José Martínez', 'Josep Martínez', 89], ['Álex Sola', 'Álex Sala', 89], ['Álex Sola', 'Álex Sala', 89], ['Álex Sola', 'Álex Sala', 89], ['Álex Sola', 'Álex Sala', 89], ['Samu Sáiz', 'Samu Saiz', 89], ['Samu Sáiz', 'Samu Saiz', 89], ['Samu Sáiz', 'Samu Saiz', 89], ['Samu Sáiz', 'Samu Saiz', 89]]
         Player  Apps  Mins Goals Assists Yel Red  SpG   PS% AerialsWon  ...  \
2258  Samu Saiz  1(7)   236     2       0   1   0  0.5  79.3        0.3  ...   

     Fouled  Off Disp UnsTch  AvgP Crosses LongB ThrB        año  Equipo  
2258    0.5  0.3  0.5    1.3  18.8     0.3   0.9  0.1  2022-2023  Girona  

[1 rows x 31 columns]


In [826]:
resultados_filtrados_88 = [elemento for elemento in results if elemento[2] == 88]
print(resultados_filtrados_88)
datos.loc[datos['Player'] == resultados_filtrados_88[3][0], 'Player'] = resultados_filtrados_88[3][1]
datos.loc[datos['Player'] == resultados_filtrados_88[6][0], 'Player'] = resultados_filtrados_88[6][1]
print(datos[datos['Player'] == resultados_filtrados_88[6][1]])

[['Carlos Fernández', 'Marcos Fernández', 88], ['Carles Pérez', 'Charlie Pérez', 88], ['Christian Rivera', 'Cristian Olivera', 88], ['Leonárdo Koútris', 'Leonardo Koutris', 88], ['Carlos Fernández', 'Marcos Fernández', 88], ['Carlos Fernández', 'Marcos Fernández', 88], ['Alexandru Tirlea', 'Alexandru Țîrlea', 88], ['Carles Pérez', 'Charlie Pérez', 88], ['Carlos Fernández', 'Marcos Fernández', 88]]
                Player  Apps  Mins Goals Assists Yel Red SpG   PS% AerialsWon  \
1516  Alexandru Țîrlea  0(1)    14     0       0   0   0   0  71.4          0   

      ... Fouled Off Disp UnsTch AvgP Crosses LongB ThrB        año  \
1516  ...      0   0    0      0    7       1     0    0  2021-2022   

                Equipo  
1516  Deportivo Alaves  

[1 rows x 31 columns]


In [827]:
resultados_filtrados_87 = [elemento for elemento in results if elemento[2] == 87]
print(resultados_filtrados_87)
datos.loc[datos['Player'] == resultados_filtrados_87[1][0], 'Player'] = resultados_filtrados_87[1][1]
print(datos[datos['Player'] == resultados_filtrados_87[1][1]])

[['James Rodríguez', 'Jesús Rodríguez', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Dani García', 'Santi García', 87], ['Dani García', 'Santi García', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Dani García', 'Santi García', 87], ['Dani García', 'Santi García', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Dani García', 'Santi García', 87], ['Dani García', 'Santi García', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Vinícius Júnior', 'Vinicius Junior', 87], ['Dani García', 'Santi García

In [828]:
resultados_filtrados_86 = [elemento for elemento in results if elemento[2] == 86]
print(resultados_filtrados_86)
datos.loc[datos['Player'] == resultados_filtrados_86[-1][0], 'Player'] = resultados_filtrados_86[-1][1]
datos.loc[datos['Player'] == resultados_filtrados_86[-3][0], 'Player'] = resultados_filtrados_86[-3][1]
print(datos[datos['Player'] == resultados_filtrados_86[-1][1]])

[['Miguel Ángel Moyá', 'Miguel Ángel Morro', 86], ['Miguel Ángel Moyá', 'Miguel Ángel Morro', 86], ['Miguel Ángel Moyá', 'Miguel Ángel Morro', 86], ['Juan Pérez', 'Julen Pérez', 86], ['Lluís López', 'Luis López', 86], ['Lluís López', 'Luis López', 86], ['Lluís López', 'Luis López', 86], ['Juan Pérez', 'Julen Pérez', 86], ['Stephane Paul Keller', 'Stephane Keller', 86]]
               Player  Apps  Mins Goals Assists Yel Red SpG   PS% AerialsWon  \
1106  Stephane Keller  0(1)    16     0       0   0   0   0  84.6          0   

      ... Fouled Off Disp UnsTch AvgP Crosses LongB ThrB        año  \
1106  ...      0   0    0      0   13       0     0    0  2020-2021   

                Equipo  
1106  Deportivo Alaves  

[1 rows x 31 columns]


In [829]:
resultados_filtrados_83 = [elemento for elemento in results if elemento[2] == 83]
datos.loc[datos['Player'] == resultados_filtrados_83[6][0], 'Player'] = resultados_filtrados_83[6][1]
datos.loc[datos['Player'] == resultados_filtrados_83[-1][0], 'Player'] = resultados_filtrados_83[-1][1]
print(print(datos[datos['Player'] == resultados_filtrados_83[6][1]]))

           Player Apps  Mins Goals Assists Yel Red SpG   PS% AerialsWon  ...  \
246  Tomas Vaclik   33  2926     0       0   0   0   0  66.5        0.2  ...   
971  Tomas Vaclik    5   450     0       0   0   0   0  80.5          0  ...   

    Fouled Off Disp UnsTch  AvgP Crosses LongB ThrB        año   Equipo  
246    0.1   0    0      0  23.4       0     5    0  2019-2020  Sevilla  
971    0.4   0    0      0  24.6       0   4.6    0  2020-2021  Sevilla  

[2 rows x 31 columns]
None


In [830]:
resultados_filtrados_82 = [elemento for elemento in results if elemento[2] == 82]
datos.loc[datos['Player'] == resultados_filtrados_82[-3][0], 'Player'] = resultados_filtrados_82[-3][1]
print(print(datos[datos['Player'] == resultados_filtrados_82[-3][1]]))

                    Player  Apps  Mins Goals Assists Yel Red SpG   PS%  \
2141  Jon Ander Olasagasti  0(3)    63     0       0   0   0   1  81.3   

     AerialsWon  ... Fouled Off Disp UnsTch AvgP Crosses LongB ThrB  \
2141        0.7  ...    1.3   0    0    0.7   16       0   0.7    0   

            año         Equipo  
2141  2022-2023  Real Sociedad  

[1 rows x 31 columns]
None


In [831]:
datos.loc[datos['Player'] == 'Nacho', 'Player'] = 'Nacho Fernández'

In [832]:
condicion = tf_data['name'].isin(datos['Player'])

# Filtrar tf_data con la condición
resultados = tf_data[~condicion]
resultados.to_csv('No_encontrados.csv')
# Imprimir los resultados
print(len(resultados))

635


In [876]:
df_concatenado = pd.merge(datos, tf_data, left_on=['Player', 'año'], right_on=['name', 'Anio'], how='left')
print(df_concatenado)

                 Player   Apps  Mins Goals Assists Yel Red  SpG   PS%  \
0          Lionel Messi  32(1)  2881    25      21   4   0  4.8  82.6   
1     Guillermo Maripán      1    90     0       0   0   0    0    75   
2              Casemiro     35  3091     4       3  12   0  1.3  84.1   
3         Karim Benzema  36(1)  3155    21       8   0   0  3.4  86.2   
4           Luis Suárez  22(6)  2002    16       8   4   0  2.8  75.5   
...                 ...    ...   ...   ...     ...  ..  ..  ...   ...   
2449     Javier Pastore   0(1)     4     0       0   0   0    0  85.7   
2450        Iván Romero   0(1)    10     0       0   1   0    0  33.3   
2451     Roger Martínez   0(1)     6     0       0   0   0    1   100   
2452     Carlos Álvarez   0(1)     9     0       0   1   0    0   100   
2453        Pape Cheikh   1(2)    36     0       0   1   1    0  73.7   

     AerialsWon  ...   age             nationality height   foot  \
0           0.3  ...  33.0  ['Argentina', 'Spain']  1,7

In [877]:
df_concatenado.keys()

Index(['Player', 'Apps', 'Mins', 'Goals', 'Assists', 'Yel', 'Red', 'SpG',
       'PS%', 'AerialsWon', 'MotM', 'Tackles', 'Inter', 'Fouls', 'Offsides',
       'Clear', 'DeffDrb', 'Blocks', 'OwnG', 'KeyP', 'OffDrb', 'Fouled', 'Off',
       'Disp', 'UnsTch', 'AvgP', 'Crosses', 'LongB', 'ThrB', 'año', 'Equipo',
       'name', 'position', 'dateOfBirth', 'age', 'nationality', 'height',
       'foot', 'joinedOn', 'signedFrom', 'marketValue', 'status', 'joined',
       'Anio'],
      dtype='object')

In [878]:
df_concatenado[df_concatenado['Player'] == 'Nacho Fernández']

,Player,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,...,age,nationality,height,foot,joinedOn,signedFrom,marketValue,status,joined,Anio
19,Nacho Fernández,4(2),438,1,0,3,0,1,86.8,2.7,...,30.0,['Spain'],"1,80m",right,"Sep 1, 2012",Real Madrid Castilla,€12.00m,Team captain,NaN,2019-2020
680,Nacho Fernández,22(2),2000,1,0,8,0,0.5,90.5,1.3,...,31.0,['Spain'],"1,80m",right,"Sep 1, 2012",Real Madrid Castilla,€10.00m,Team captain,NaN,2020-2021
1391,Nacho Fernández,17(11),1549,3,0,7,0,0.4,93,1,...,32.0,['Spain'],"1,80m",right,"Sep 1, 2012",Real Madrid Castilla,€7.00m,Team captain,NaN,2021-2022
2034,Nacho Fernández,18(9),1632,1,1,7,0,0.3,93,1.3,...,33.0,['Spain'],"1,80m",right,"Sep 1, 2012",Real Madrid Castilla,€6.00m,Team captain,NaN,2022-2023


In [879]:
df_concatenado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2454 entries, 0 to 2453
Data columns (total 44 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Player       2454 non-null   object 
 1   Apps         2454 non-null   object 
 2   Mins         2454 non-null   int64  
 3   Goals        2454 non-null   object 
 4   Assists      2454 non-null   object 
 5   Yel          2454 non-null   object 
 6   Red          2454 non-null   object 
 7   SpG          2454 non-null   object 
 8   PS%          2454 non-null   object 
 9   AerialsWon   2454 non-null   object 
 10  MotM         2454 non-null   object 
 11  Tackles      2454 non-null   object 
 12  Inter        2454 non-null   object 
 13  Fouls        2454 non-null   object 
 14  Offsides     2454 non-null   object 
 15  Clear        2454 non-null   object 
 16  DeffDrb      2454 non-null   object 
 17  Blocks       2454 non-null   object 
 18  OwnG         2454 non-null   object 
 19  KeyP  

In [880]:
df_concatenado = df_concatenado.dropna(subset=['marketValue'], axis=0)
df_concatenado = df_concatenado.drop(['name','Anio'],axis = 1)
df_concatenado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1916 entries, 0 to 2452
Data columns (total 42 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Player       1916 non-null   object 
 1   Apps         1916 non-null   object 
 2   Mins         1916 non-null   int64  
 3   Goals        1916 non-null   object 
 4   Assists      1916 non-null   object 
 5   Yel          1916 non-null   object 
 6   Red          1916 non-null   object 
 7   SpG          1916 non-null   object 
 8   PS%          1916 non-null   object 
 9   AerialsWon   1916 non-null   object 
 10  MotM         1916 non-null   object 
 11  Tackles      1916 non-null   object 
 12  Inter        1916 non-null   object 
 13  Fouls        1916 non-null   object 
 14  Offsides     1916 non-null   object 
 15  Clear        1916 non-null   object 
 16  DeffDrb      1916 non-null   object 
 17  Blocks       1916 non-null   object 
 18  OwnG         1916 non-null   object 
 19  KeyP  

In [881]:
extracted = df_concatenado['Apps'].str.extract(r'(\d+)?\((\d+)?\)')

# Asignar valores predeterminados de 0 cuando no hay paréntesis o solo hay un número
df_concatenado['Titularidades'] = extracted[0].fillna(0)
df_concatenado['Suplencias'] = extracted[1].fillna(0)

# Convertir a enteros
df_concatenado['Titularidades'] = df_concatenado['Titularidades'].astype(int)
df_concatenado['Suplencias'] = df_concatenado['Suplencias'].astype(int)
df_concatenado = df_concatenado.drop('Apps',axis = 1)

In [882]:
df_concatenado['height'][0].split('m')[0]

'1,70'

In [883]:
def altura(x):
    if isinstance(x, str):
        return x.split('m')[0]
    else:
        return x

In [884]:
def convertir_abreviaturas(valor):
    valor_sin_euro = valor.replace('€', '')
    if 'm' in valor:
        return round(float(valor_sin_euro.replace('m', '')) * 1000000, 2)  # Multiplicar por 1 millón
    elif 'k' in valor:
        return round(float(valor_sin_euro.replace('k', '')) * 1000, 2)  # Multiplicar por 1000
    else:
        return round(float(valor_sin_euro), 2)

# Aplicar la función a la columna 'marketValue'
df_concatenado['marketValue'] = df_concatenado['marketValue'].apply(convertir_abreviaturas)


In [885]:
df_concatenado

,Player,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,...,nationality,height,foot,joinedOn,signedFrom,marketValue,status,joined,Titularidades,Suplencias
0,Lionel Messi,2881,25,21,4,0,4.8,82.6,0.3,22,...,"['Argentina', 'Spain']","1,70m",left,"Jul 1, 2005",FC Barcelona B,112000000.0,NaN,NaN,32,1
1,Guillermo Maripán,90,0,0,0,0,0,75,5,0,...,['Chile'],"1,93m",right,"Jul 7, 2017",CD Universidad Católica,11000000.0,NaN,NaN,0,0
2,Casemiro,3091,4,3,12,0,1.3,84.1,2.9,3,...,"['Brazil', 'Spain']","1,85m",right,"Jul 2, 2015",FC Porto,64000000.0,NaN,NaN,0,0
3,Karim Benzema,3155,21,8,0,0,3.4,86.2,0.8,10,...,"['France', 'Algeria']","1,85m",right,"Jul 9, 2009",: Ablöse €35.00m,32000000.0,NaN,NaN,36,1
4,Luis Suárez,2002,16,8,4,0,2.8,75.5,0.4,3,...,['Uruguay'],"1,82m",right,"Jul 11, 2014",Liverpool FC,28000000.0,NaN,NaN,22,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2445,Mario Suárez,11,0,0,0,1,0,66.7,0,0,...,['Spain'],"1,89m",right,"Jan 31, 2019",Guizhou Hengfeng,300000.0,NaN,NaN,1,2
2446,Pol Lozano,13,0,0,1,0,0,71.4,0.3,0,...,['Spain'],"1,76m",right,"Jan 30, 2023",RCD Espanyol Barcelona,1500000.0,NaN,NaN,0,4
2447,Fer Niño,19,0,0,0,0,0.3,100,0,0,...,['Spain'],"1,91m",right,"Aug 1, 2020",Villarreal CF B,3000000.0,NaN,NaN,0,3
2450,Iván Romero,10,0,0,1,0,0,33.3,1,0,...,['Spain'],"1,73m",right,"Jul 1, 2022",Sevilla Atlético,800000.0,NaN,Returned after loan spell with CD Tenerife; da...,0,1


In [886]:
df_concatenado['height'] = df_concatenado['height'].map(altura)
df_concatenado = df_concatenado.reset_index(drop=True)
df_concatenado

,Player,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MotM,...,nationality,height,foot,joinedOn,signedFrom,marketValue,status,joined,Titularidades,Suplencias
0,Lionel Messi,2881,25,21,4,0,4.8,82.6,0.3,22,...,"['Argentina', 'Spain']","1,70",left,"Jul 1, 2005",FC Barcelona B,112000000.0,NaN,NaN,32,1
1,Guillermo Maripán,90,0,0,0,0,0,75,5,0,...,['Chile'],"1,93",right,"Jul 7, 2017",CD Universidad Católica,11000000.0,NaN,NaN,0,0
2,Casemiro,3091,4,3,12,0,1.3,84.1,2.9,3,...,"['Brazil', 'Spain']","1,85",right,"Jul 2, 2015",FC Porto,64000000.0,NaN,NaN,0,0
3,Karim Benzema,3155,21,8,0,0,3.4,86.2,0.8,10,...,"['France', 'Algeria']","1,85",right,"Jul 9, 2009",: Ablöse €35.00m,32000000.0,NaN,NaN,36,1
4,Luis Suárez,2002,16,8,4,0,2.8,75.5,0.4,3,...,['Uruguay'],"1,82",right,"Jul 11, 2014",Liverpool FC,28000000.0,NaN,NaN,22,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1911,Mario Suárez,11,0,0,0,1,0,66.7,0,0,...,['Spain'],"1,89",right,"Jan 31, 2019",Guizhou Hengfeng,300000.0,NaN,NaN,1,2
1912,Pol Lozano,13,0,0,1,0,0,71.4,0.3,0,...,['Spain'],"1,76",right,"Jan 30, 2023",RCD Espanyol Barcelona,1500000.0,NaN,NaN,0,4
1913,Fer Niño,19,0,0,0,0,0.3,100,0,0,...,['Spain'],"1,91",right,"Aug 1, 2020",Villarreal CF B,3000000.0,NaN,NaN,0,3
1914,Iván Romero,10,0,0,1,0,0,33.3,1,0,...,['Spain'],"1,73",right,"Jul 1, 2022",Sevilla Atlético,800000.0,NaN,Returned after loan spell with CD Tenerife; da...,0,1


In [887]:
df_concatenado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 43 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Player         1916 non-null   object 
 1   Mins           1916 non-null   int64  
 2   Goals          1916 non-null   object 
 3   Assists        1916 non-null   object 
 4   Yel            1916 non-null   object 
 5   Red            1916 non-null   object 
 6   SpG            1916 non-null   object 
 7   PS%            1916 non-null   object 
 8   AerialsWon     1916 non-null   object 
 9   MotM           1916 non-null   object 
 10  Tackles        1916 non-null   object 
 11  Inter          1916 non-null   object 
 12  Fouls          1916 non-null   object 
 13  Offsides       1916 non-null   object 
 14  Clear          1916 non-null   object 
 15  DeffDrb        1916 non-null   object 
 16  Blocks         1916 non-null   object 
 17  OwnG           1916 non-null   object 
 18  KeyP    

In [888]:
df_concatenado[['Mins','Goals','Assists','Yel','Red','MotM','OwnG','age']] = df_concatenado[['Mins','Goals','Assists','Yel','Red','MotM','OwnG','age']].astype(int)
df_concatenado[['SpG','PS%','AerialsWon','Tackles','Inter','Fouls','Offsides','Clear','DeffDrb','Blocks','KeyP','OffDrb','Fouled','Off','Disp','UnsTch','AvgP','Crosses','LongB','ThrB']] = df_concatenado[['SpG','PS%','AerialsWon','Tackles','Inter','Fouls','Offsides','Clear','DeffDrb','Blocks','KeyP','OffDrb','Fouled','Off','Disp','UnsTch','AvgP','Crosses','LongB','ThrB']].astype(float)


In [889]:
df_concatenado['height'] = df_concatenado['height'].str.replace(',', '.').astype(float)

In [890]:
df_concatenado = df_concatenado.drop(['dateOfBirth','foot'],axis=1)

In [891]:
df_concatenado['Equipo'].unique()

array(['Barcelona', 'Deportivo Alaves', 'Real Madrid', 'Real Sociedad',
       'Villarreal', 'Real Betis', 'Atletico', 'Sevilla', 'Athletic Club',
       'Osasuna', 'Celta Vigo', 'Valencia', 'Espanyol', 'Granada',
       'Mallorca', 'Getafe', 'Leganes', 'Levante', 'Real Valladolid',
       'Eibar', 'Cadiz', 'SD Huesca', 'Rayo Vallecano', 'Elche', 'Girona',
       'Almeria'], dtype=object)

In [892]:
data_2022_nombres = {
    'Barcelona': 1,
    'Real Madrid': 2,
    'Atletico': 3,
    'Real Sociedad': 4,
    'Villarreal': 5,
    'Real Betis': 6,
    'Osasuna': 7,
    'Athletic Club': 8,
    'Mallorca': 9,
    'Girona': 10,
    'Sevilla': 11,
    'Rayo Vallecano': 12,
    'Celta Vigo': 13,
    'Valencia': 14,
    'Getafe': 15,
    'Cadiz': 16,
    'Almeria': 17,
    'Real Valladolid': 18,
    'Espanyol': 19,
    'Elche': 20,
    'Granada': 21,
    'Levante': 22,
    'Deportivo Alaves': 23,
    'Leganes':24,
    'SD Huesca': 25,
    'Eibar': 26,
}

data_2021_nombres = {
    'Real Madrid': 1,
    'Barcelona': 2,
    'Atletico': 3,
    'Sevilla': 4,
    'Real Betis': 5,
    'Real Sociedad': 6,
    'Villarreal': 7,
    'Athletic Club': 8,
    'Valencia': 9,
    'Osasuna': 10,
    'Celta Vigo': 11,
    'Rayo Vallecano': 12,
    'Elche': 13,
    'Espanyol': 14,
    'Getafe': 15,
    'Mallorca': 16,
    'Cadiz': 17,
    'Granada': 18,
    'Levante': 19,
    'Deportivo Alaves': 20,
    'SD Huesca': 21,
    'Real Valladolid': 22,
    'Eibar': 23,
    'Girona': 24,
    'Leganes': 25,
    'Almeria': 26
}

data_2020_nombres = {
    'Atletico': 1,
    'Real Madrid': 2,
    'Barcelona': 3,
    'Sevilla': 4,
    'Real Sociedad': 5,
    'Real Betis': 6,
    'Villarreal': 7,
    'Celta Vigo': 8,
    'Athletic Club': 9,
    'Granada': 10,
    'Osasuna': 11,
    'Cadiz': 12,
    'Valencia': 13,
    'Levante': 14,
    'Getafe': 15,
    'Deportivo Alaves': 16,
    'Elche': 17,
    'SD Huesca': 18,
    'Real Valladolid': 19,
    'Eibar': 20,
    'Leganes': 21,
    'Mallorca': 22,
    'Espanyol': 23,
    'Girona': 24,
    'Rayo Vallecano': 25,
    'Almeria': 26
}

data_2019_nombres = {
    'Real Madrid': 1,
    'Barcelona': 2,
    'Atletico': 3,
    'Sevilla': 4,
    'Villarreal': 5,
    'Real Sociedad': 6,
    'Granada': 7,
    'Getafe': 8,
    'Valencia': 9,
    'Osasuna': 10,
    'Athletic Club': 11,
    'Levante': 12,
    'Real Valladolid': 13,
    'Eibar': 14,
    'Real Betis': 15,
    'Deportivo Alaves': 16,
    'Celta Vigo': 17,
    'Leganes': 18,
    'Mallorca': 19,
    'Espanyol': 20,
    'SD Huesca': 21,   
    'Cadiz': 22,    
    'Rayo Vallecano': 23, 
    'Girona': 24,  
    'Almeria': 25,  
    'Elche': 26
}

In [893]:
df_concatenado['equipo_2022'] = df_concatenado['Equipo'].map(data_2022_nombres)
df_concatenado['equipo_2021'] = df_concatenado['Equipo'].map(data_2021_nombres)
df_concatenado['equipo_2020'] = df_concatenado['Equipo'].map(data_2020_nombres)
df_concatenado['equipo_2019'] = df_concatenado['Equipo'].map(data_2019_nombres)
df_concatenado = df_concatenado.drop('Equipo', axis = 1)
df_concatenado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1916 entries, 0 to 1915
Data columns (total 44 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Player         1916 non-null   object 
 1   Mins           1916 non-null   int64  
 2   Goals          1916 non-null   int64  
 3   Assists        1916 non-null   int64  
 4   Yel            1916 non-null   int64  
 5   Red            1916 non-null   int64  
 6   SpG            1916 non-null   float64
 7   PS%            1916 non-null   float64
 8   AerialsWon     1916 non-null   float64
 9   MotM           1916 non-null   int64  
 10  Tackles        1916 non-null   float64
 11  Inter          1916 non-null   float64
 12  Fouls          1916 non-null   float64
 13  Offsides       1916 non-null   float64
 14  Clear          1916 non-null   float64
 15  DeffDrb        1916 non-null   float64
 16  Blocks         1916 non-null   float64
 17  OwnG           1916 non-null   int64  
 18  KeyP    

In [894]:
df_concatenado.to_csv('Datos finales LaLiga.csv')

In [853]:
tf_data.to_csv('Transfermarket limpio.csv')